In [1]:
ljy_fp = r"C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\objdump-D\-dmanytest.asm"
GuGuJi_fp = r"/Users/gugujixiao/allsync/hw_benchmark/LBM/try.asm"
GuGuJi_oldfp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/hw-memory/benchmarks/old_benchmark/spec_example/spec2006_470.lbm/lbm_part.asm"
fp = GuGuJi_fp

In [2]:
'''读指令, 指令对象化'''
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

        

for i in temp_stat:
     if i.is_ls:
        print(i.tokens,i.ls_reg_target,i.ls_reg_target_list,i.ls_addr_mode)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
print()

here
('x0', '', 'x1', 'lsl', '', '2')
here
('x0', '', 'x1', 'lsl', '', '2')
('40049c', 'a9bf7bfd', 'stp', None, 'x29, x30, [sp, #-16]!') None [] None
('4004a8', 'a8c17bfd', 'ldp', None, 'x29, x30, [sp], #16') None [] None
('4004b0', 'a9bf7bf0', 'stp', None, 'x16, x30, [sp, #-16]!') None [] None
('4004b8', 'f947fe11', 'ldr', None, 'x17, [x16, #4088]') x16 [] AddrMode.ImmeOffset
('4004d4', 'f9400211', 'ldr', None, 'x17, [x16]') x16 [] AddrMode.Base
('4004e4', 'f9400611', 'ldr', None, 'x17, [x16, #8]') x16 [] AddrMode.ImmeOffset
('4004f4', 'f9400a11', 'ldr', None, 'x17, [x16, #16]') x16 [] AddrMode.ImmeOffset
('400510', 'f94003e1', 'ldr', None, 'x1, [sp]') sp [] AddrMode.Base
('400548', 'f947ec00', 'ldr', None, 'x0, [x0, #4056]') x0 [] AddrMode.ImmeOffset
('40057c', 'f947e821', 'ldr', None, 'x1, [x1, #4048]') x1 [] AddrMode.ImmeOffset
('4005b8', 'f947f042', 'ldr', None, 'x2, [x2, #4064]') x2 [] AddrMode.ImmeOffset
('4005d0', 'a9be7bfd', 'stp', None, 'x29, x30, [sp, #-32]!') None [] None
(

In [3]:
'''分proc'''
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
'''处理call_graph'''
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
'''生成TCFG和loop信息'''
from newCFG.cfg import TCfg
import random

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




In [6]:
'''设置node和edge上的的value'''

for l in tcfg.loops:
    l.back_edge.is_backEdge = True
for n in tcfg_nodes:
    n.set_rw_data()

temp_bool = True

while temp_bool:
    no_end = False

    for n in tcfg_nodes:
        n.set_rw_value()
        if not n.no_out:
            no_end = True
        temp_bool = no_end

for e in tcfg_edges:
    if e.is_backEdge:
        e.loop_value = random.randint(0,10)
        e.edge_value = 2

for e in tcfg_edges:
    print(e.src.name,e.dst.name,e.edge_value)

n0 n2 1.0
n1 n2 2
n2 n1 0.5
n2 n3 0.5


In [7]:
'''ls分析和读写特征'''

from newCFG.cfg import TCfg
from newCFG.loadstore import LSProc
from newCFG.rw_condition import RWProc, RWType

lsproc = LSProc(tcfg_nodes)

lds_table = lsproc.ls_table

for i in lds_table:
    i.final_addr
    i.local_offset

#for i in lds_table:
    #print(i.ins.tokens,i.ins.final_addr)

rwproc = RWProc(lds_table)

Global_Tolerant_value = 0
Global_Intolerant_value = 0

for rwu in rwproc.rw_table:
    #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
    if rwu.is_torrent == RWType.Global_Tolerant:
        Global_Tolerant_value += rwu.find_cycle
        print(rwu.ins.tokens,rwu.ins.final_addr,rwu.ls_reg_target)
    if rwu.is_torrent == RWType.Global_Intolerant:
        Global_Intolerant_value += rwu.find_cycle
        print(rwu.ins.tokens,rwu.ins.final_addr,rwu.ls_reg_target)

print("全局的容错路径为：",Global_Tolerant_value)
print("全局的非容错路径为：",Global_Intolerant_value)   

AttributeError: 'RWUnit' object has no attribute 'ls_reg_target'

In [ ]:
'''cache分析'''

#打印给cache分许需要的数据
for i in lds_table:
    print("指令",i.ins.tokens,"是否是SP",i.is_sp,"指令地址",i.ins.addr.val(),"访存地址",i.final_addr,"数据宽度",i.ins.ls_data_width,i.ins.ls_addr_mode)


指令 ('400608', 'b9000fe0', 'str', None, 'w0, [sp, #12]') 是否是SP True 指令地址 4195848 访存地址 0 数据宽度 32 AddrMode.ImmeOffset
指令 ('40060c', 'f90003e1', 'str', None, 'x1, [sp]') 是否是SP True 指令地址 4195852 访存地址 0 数据宽度 32 AddrMode.Base
指令 ('400610', 'b9001fff', 'str', None, 'wzr, [sp, #28]') 是否是SP True 指令地址 4195856 访存地址 0 数据宽度 32 AddrMode.ImmeOffset
指令 ('400620', 'b9801fe1', 'ldrsw', None, 'x1, [sp, #28]') 是否是SP True 指令地址 4195872 访存地址 0 数据宽度 32 AddrMode.ImmeOffset
指令 ('40062c', 'b8217802', 'str', None, 'w2, [x0, x1, lsl #2]') 是否是SP False 指令地址 4195884 访存地址 0 数据宽度 32 AddrMode.RegShift
指令 ('400630', 'b9401fe0', 'ldr', None, 'w0, [sp, #28]') 是否是SP True 指令地址 4195888 访存地址 0 数据宽度 32 AddrMode.ImmeOffset
指令 ('400638', 'b9001fe0', 'str', None, 'w0, [sp, #28]') 是否是SP True 指令地址 4195896 访存地址 0 数据宽度 32 AddrMode.ImmeOffset
指令 ('40063c', 'b9401fe0', 'ldr', None, 'w0, [sp, #28]') 是否是SP True 指令地址 4195900 访存地址 0 数据宽度 32 AddrMode.ImmeOffset


In [ ]:
from cache_analysis.read_segment import segmentReader
from newCFG.heat_analysis import loop_heat
bread = segmentReader(r'C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\objdump-D\-D manytest.asm')
test=loop_heat(tcfg,lsproc,bread)
test.loop_heat_analysis()



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\13377\\Desktop\\hw-memory-master (1)\\hw-memory-master\\benchmarks\\objdump-D\\-D manytest.asm'